In [ ]:
import os

import joblib
from azureml.core import Dataset, Environment, Experiment, Workspace, ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import Pipeline, PipelineEndpoint, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

In [ ]:
ws = Workspace.from_config()

aml_compute_target = "clusteraml"

try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except ComputeTargetException:
    print("Creating new compute target: {}".format(aml_compute_target))
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 1)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)


In [ ]:
CustomerInference_dataset = Dataset.get_by_name(ws, name='customer_tabular')
CustomerInference_pipeline_param = PipelineParameter(name="CustomerInference_pipeline_param", default_value=CustomerInference_dataset)
CustomerInference_ds_consumption = DatasetConsumptionConfig("CustomerInference_dataset", CustomerInference_pipeline_param)

OutputPathParameter = PipelineParameter(name="output_path", default_value="CustomerIncomeOutput/CustomerIncomeOutput.csv")
OutputDatastoreParameter = PipelineParameter(name="output_datastore", default_value="workspaceblobstore")

env = Environment.from_conda_specification(name = 'env', file_path = './CustomerIncomeInferencePipeline.yml')

run_config = RunConfiguration()
run_config.environment = env

scoring_step = PythonScriptStep(
    name="Scoring_Step",
    script_name='score.py',
    arguments=["--input_data", CustomerInference_ds_consumption, "--output_path", OutputPathParameter, "--output_datastore", OutputDatastoreParameter],
    inputs=[CustomerInference_ds_consumption],
    compute_target=aml_compute_target,
    source_directory='./scripts',
    runconfig = run_config,
    allow_reuse=False)

print("Scoring step created")

pipeline = Pipeline(workspace=ws, steps=[scoring_step])

print("Pipeline is built")

In [ ]:
pipeline_endpoint_name = 'IncomeEndpoint14'
experiment_name = 'ci-aml-custom-models'

pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name=pipeline_endpoint_name, pipeline=pipeline, description="Income Endpoint")
published_pipeline = pipeline.publish(name=pipeline_endpoint_name, description="published pipeline")
pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name=pipeline_endpoint_name) 
pipeline_endpoint.add_default(pipeline=published_pipeline)
published_pipeline

In [ ]:
experiment = Experiment(ws, experiment_name)

pipeline_run = experiment.submit(pipeline_endpoint)

pipeline_run.get_details()

In [ ]:
pipeline_run.wait_for_completion(show_output=True)
